In [1]:
!pip install tensorflow gym keras-rl2 gym[atari]
%pip install -U gym
%pip install -U gym[atari,accept-rom-license]
%pip install -U gym[ale,accept-rom-license]
%pip install gym[atari]
%pip install autorom[accept-rom-license]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import gym
import random

In [2]:
env=gym.make('ALE/SpaceInvaders-v5',render_mode='human')
height,width,channels=env.observation_space.shape
actions=env.action_space.n

C:\Users\user\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


In [3]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [14]:
episodes=5
for episode in range(1,episodes+1):
    state = env.reset()
    done = False
    score=0
    
    while not done:
        env.render(mode="rgb_array")
        action=random.choice([0,1,2,3,4,5])
        n_state,reward,done,info=env.step(action)
        score+=reward
    print("Episode:{} Score:{} ".format(episode,score))
env.close()

C:\Users\user\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:305: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps
  logger.warn(


Episode:1 Score:90.0 
Episode:2 Score:210.0 
Episode:3 Score:215.0 
Episode:4 Score:135.0 
Episode:5 Score:180.0 


In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Convolution2D
from tensorflow.keras.optimizers import Adam

C:\Users\user\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\user\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
C:\Users\user\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:29: DeprecationWarning: Call to deprecated create function Descriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _TENSORSHAPEPROTO_DIM

In [4]:
def build_model(height, width, channels, actions):
    model=Sequential()
    model.add(Convolution2D(32,(8,8), strides=(4,4),activation='relu', input_shape=(3,height,width,channels)))
    model.add(Convolution2D(64,(4,4), strides=(2,2),activation='relu'))
    model.add(Convolution2D(64,(3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model

In [8]:
del model

In [9]:
model=build_model(height,width,channels,actions)

In [10]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy,EpsGreedyQPolicy

In [11]:
def build_agent(model,actions):
    policy=LinearAnnealedPolicy(EpsGreedyQPolicy(),attr='eps',value_max=1.,value_min=.1,value_test=.2,nb_steps=10000)
    memory=SequentialMemory(limit=1000,window_length=3)
    dqn=DQNAgent(model=model,memory=memory,policy=policy,enable_dueling_network=True,dueling_type='avg',
                 nb_actions=actions,nb_steps_warmup=1000)
    return dqn

In [14]:
dqn=build_agent(model,actions)
dqn.compile(Adam(lr=1e-4))
dqn.fit(env,nb_steps=10000,visualize=False,verbose=2)

C:\Users\user\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 10000 steps ...


C:\Users\user\anaconda3\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  417/10000: episode: 1, duration: 62.315s, episode steps: 417, steps per second:   7, episode reward: 120.000, mean reward:  0.288 [ 0.000, 30.000], mean action: 2.465 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  906/10000: episode: 2, duration: 73.304s, episode steps: 489, steps per second:   7, episode reward: 155.000, mean reward:  0.317 [ 0.000, 30.000], mean action: 2.644 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --


C:\Users\user\anaconda3\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 1519/10000: episode: 3, duration: 1772.334s, episode steps: 613, steps per second:   0, episode reward: 325.000, mean reward:  0.530 [ 0.000, 200.000], mean action: 2.494 [0.000, 5.000],  loss: 9.598262, mean_q: 3.988221, mean_eps: 0.886645
 1883/10000: episode: 4, duration: 1127.047s, episode steps: 364, steps per second:   0, episode reward: 55.000, mean reward:  0.151 [ 0.000, 20.000], mean action: 2.286 [0.000, 5.000],  loss: 12.072577, mean_q: 5.009389, mean_eps: 0.846955
 2367/10000: episode: 5, duration: 1498.338s, episode steps: 484, steps per second:   0, episode reward: 165.000, mean reward:  0.341 [ 0.000, 30.000], mean action: 2.378 [0.000, 5.000],  loss: 1.985317, mean_q: 4.298465, mean_eps: 0.808795
 3140/10000: episode: 6, duration: 2293.745s, episode steps: 773, steps per second:   0, episode reward: 260.000, mean reward:  0.336 [ 0.000, 30.000], mean action: 2.678 [0.000, 5.000],  loss: 0.906347, mean_q: 3.418628, mean_eps: 0.752230
 3800/10000: episode: 7, duration: 

In [15]:
dqn.save_weights('dqn_weights.h5f')

In [12]:
dqn=build_agent(model,actions)
dqn.compile(Adam(lr=1e-4))
dqn.load_weights('10k/dqn_weights.h5f')

C:\Users\user\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [15]:
scores=dqn.test(env,nb_episodes=10,visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 55.000, steps: 426
Episode 2: reward: 10.000, steps: 249
Episode 3: reward: 120.000, steps: 565
Episode 4: reward: 25.000, steps: 495
Episode 5: reward: 370.000, steps: 616
Episode 6: reward: 50.000, steps: 406
Episode 7: reward: 20.000, steps: 272
Episode 8: reward: 240.000, steps: 896
Episode 9: reward: 40.000, steps: 299
Episode 10: reward: 5.000, steps: 254
93.5
